In [1]:
param_market = 'DE'

In [2]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [3]:
input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
list_job

总数量：1


[(0, 0)]

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

In [6]:
a = 0

i, j = list_job[a]

data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

# ua = UA.get_User_Agent_Requests()
# ua['Content-Type'] = 'application/json'
ua = {'Content-Type': 'application/json'}

if param_market == 'US':
    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
elif param_market == 'DE':
    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
elif param_market == 'AU':
    url = 'https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&ul=AU&_stpos=2019&orig_cvip=true'
elif param_market == 'UK':
    url = 'https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=GB&_stpos=BS81QU&orig_cvip=true'

resp = requests.post(url,
                     data=data,
                     headers=ua,
                     proxies=Proxy.get_Proxy_Requests()).text

resp

'{"modules":{"COMPATIBILITY_TABLE":{"_type":"CompatibilityTableModule","paginatedTable":{"_type":"PaginatedTable","title":{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Teil ist kompatibel mit 19 Fahrzeug(en)"}]},"header":{"_type":"TableHeader","cells":[{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Marke"}]},{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Modell"}]},{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Baujahr"}]},{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Plattform"}]},{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Typ"}]},{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Motor"}]},{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Einschränkungen"}]}]},"caption":{"_type":"TextualDisplay","textSpans":[{"_type":"TextSpan","text":"Einige der in dieser Tabelle aufgeführten Informationen wurden bereitgestellt von null"}]},"r

In [7]:
import json

In [8]:
json_data = json.loads(resp)

json_data

{'modules': {'COMPATIBILITY_TABLE': {'_type': 'CompatibilityTableModule',
   'paginatedTable': {'_type': 'PaginatedTable',
    'title': {'_type': 'TextualDisplay',
     'textSpans': [{'_type': 'TextSpan',
       'text': 'Teil ist kompatibel mit 19 Fahrzeug(en)'}]},
    'header': {'_type': 'TableHeader',
     'cells': [{'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Marke'}]},
      {'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Modell'}]},
      {'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Baujahr'}]},
      {'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Plattform'}]},
      {'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Typ'}]},
      {'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Motor'}]},
      {'_type': 'TextualDisplay',
       'textSpans': [{'_type': 'TextSpan', 'text': 'Einschränkung

In [9]:
list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']

list_header

[{'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Marke'}]},
 {'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Modell'}]},
 {'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Baujahr'}]},
 {'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Plattform'}]},
 {'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Typ'}]},
 {'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Motor'}]},
 {'_type': 'TextualDisplay',
  'textSpans': [{'_type': 'TextSpan', 'text': 'Einschränkungen'}]}]

In [10]:
col_engine = -1
if param_market in ['US', 'AU', 'UK']:
    name_make = 'Make'
    name_model = 'Model'
    name_engine = 'Engine'
    name_year = 'Year'
elif param_market == 'DE':
    name_make = 'Marke'
    name_model = 'Modell'
    name_engine = 'Motor'
    name_year = 'Baujahr'
    
for k in range(len(list_header)):
    if name_make in list_header[k]['textSpans'][0]['text']:
        col_make = k
    elif name_model in list_header[k]['textSpans'][0]['text']:
        col_model = k
    elif name_engine in list_header[k]['textSpans'][0]['text']:
        col_engine = k
    elif name_year in list_header[k]['textSpans'][0]['text']:
        col_year = k
        
col_make, col_model, col_engine, col_year

(0, 1, 5, 2)

In [11]:
list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

list_row

[{'_type': 'TableRow',
  'cells': [{'_type': 'TextualDisplay',
    'textSpans': [{'_type': 'TextSpan', 'text': 'Toyota'}]},
   {'_type': 'TextualDisplay',
    'textSpans': [{'_type': 'TextSpan', 'text': 'Corolla'}]},
   {'_type': 'TextualDisplay',
    'textSpans': [{'_type': 'TextSpan', 'text': '2004/07-2007/02'}]},
   {'_type': 'TextualDisplay',
    'textSpans': [{'_type': 'TextSpan', 'text': 'E12'}]},
   {'_type': 'TextualDisplay',
    'textSpans': [{'_type': 'TextSpan', 'text': '1.4 D'}]},
   {'_type': 'TextualDisplay',
    'textSpans': [{'_type': 'TextSpan', 'text': '1364 ccm, 66 KW, 90 PS'}]},
   {'_type': 'ExpandableTextualDisplayBlock',
    'textualDisplays': [{'_type': 'TextualDisplay',
      'textSpans': [{'_type': 'TextSpan',
        'text': 'Fahrzeugproduktionsland Europa | Fahrzeugproduktionsland Japan'}]}],
    'expandCollapseControls': {'_type': 'ExpandCollapseControls',
     'viewMore': {'_type': 'TextualDisplay',
      'textSpans': [{'_type': 'TextSpan',
        'text':

In [12]:
for row in list_row:
    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
    if col_engine == -1:
        engine = ''
    else:
        engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
    if param_market in ['US', 'AU', 'UK']:
        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
    elif param_market == 'DE':
        list_year = [int(year.split('/')[0].strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]
        
    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
    else:
        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
        else:
            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
            else:
                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year
                
    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = sorted(list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine])))
    
dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1
    
dict_item[input_.loc[i, 'Item_Number']]

{'vehicle_page': 1,
 'crawler': 1,
 'dict_vehicle': {'Toyota': {'Corolla': {'1364 ccm, 66 KW, 90 PS': [2004,
     2007],
    '1398 ccm, 71 KW, 97 PS': [2001, 2007],
    '1598 ccm, 81 KW, 110 PS': [2002, 2006],
    '1796 ccm, 141 KW, 192 PS': [2001, 2007],
    '1796 ccm, 160 KW, 218 PS': [2005, 2006],
    '1796 ccm, 165 KW, 224 PS': [2005, 2007],
    '1995 ccm, 66 KW, 90 PS': [2002, 2006],
    '1995 ccm, 81 KW, 110 PS': [2002, 2006],
    '1995 ccm, 85 KW, 116 PS': [2003, 2007]},
   'Corolla Kombi': {'1364 ccm, 66 KW, 90 PS': [2004, 2007],
    '1398 ccm, 71 KW, 97 PS': [2002, 2007],
    '1598 ccm, 81 KW, 110 PS': [2002, 2007],
    '1995 ccm, 66 KW, 90 PS': [2002, 2007],
    '1995 ccm, 85 KW, 116 PS': [2004, 2007]},
   'Corolla Stufenheck': {'1364 ccm, 66 KW, 90 PS': [2004, 2006],
    '1398 ccm, 71 KW, 97 PS': [2001, 2006],
    '1598 ccm, 81 KW, 110 PS': [2001, 2008],
    '1995 ccm, 66 KW, 90 PS': [2002, 2006],
    '1995 ccm, 85 KW, 116 PS': [2004, 2007]}}},
 'vehicle_1': '',
 'vehicle_2'

In [13]:
dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']

True

In [14]:
list_vehicle = []
for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
    for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
        list_year = []
        for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
            list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
        list_year = sorted(list(set(list_year)))
        
        year_min = min(list_year)
        year_max = max(list_year)
        if year_min == year_max:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(sorted(list_vehicle))

print(dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'])

Toyota Corolla 2001-2007
Toyota Corolla Kombi 2002-2007
Toyota Corolla Stufenheck 2001-2008


In [15]:
list_vehicle = []
for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
    for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
        for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
            year_min = min(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine])
            year_max = max(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine])
            if year_min == year_max:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
            else:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(sorted(list_vehicle))

print(dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'])

Toyota Corolla 2001-2007 1398 ccm, 71 KW, 97 PS
Toyota Corolla 2001-2007 1796 ccm, 141 KW, 192 PS
Toyota Corolla 2002-2006 1598 ccm, 81 KW, 110 PS
Toyota Corolla 2002-2006 1995 ccm, 66 KW, 90 PS
Toyota Corolla 2002-2006 1995 ccm, 81 KW, 110 PS
Toyota Corolla 2003-2007 1995 ccm, 85 KW, 116 PS
Toyota Corolla 2004-2007 1364 ccm, 66 KW, 90 PS
Toyota Corolla 2005-2006 1796 ccm, 160 KW, 218 PS
Toyota Corolla 2005-2007 1796 ccm, 165 KW, 224 PS
Toyota Corolla Kombi 2002-2007 1398 ccm, 71 KW, 97 PS
Toyota Corolla Kombi 2002-2007 1598 ccm, 81 KW, 110 PS
Toyota Corolla Kombi 2002-2007 1995 ccm, 66 KW, 90 PS
Toyota Corolla Kombi 2004-2007 1364 ccm, 66 KW, 90 PS
Toyota Corolla Kombi 2004-2007 1995 ccm, 85 KW, 116 PS
Toyota Corolla Stufenheck 2001-2006 1398 ccm, 71 KW, 97 PS
Toyota Corolla Stufenheck 2001-2008 1598 ccm, 81 KW, 110 PS
Toyota Corolla Stufenheck 2002-2006 1995 ccm, 66 KW, 90 PS
Toyota Corolla Stufenheck 2004-2006 1364 ccm, 66 KW, 90 PS
Toyota Corolla Stufenheck 2004-2007 1995 ccm, 85 K

In [16]:
df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                         'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                         'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [17]:
crawler_status = 'ok'

crawler_status

'ok'

In [18]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                             'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                             'Category_Id': input_.loc[i, 'Category_Id'],
                             'Market_Id': input_.loc[i, 'Market_Id']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [19]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,Item_Number,Vehicle_1,Vehicle_2
0,315281840098,Toyota Corolla 2001-2007\nToyota Corolla Kombi...,"Toyota Corolla 2001-2007 1398 ccm, 71 KW, 97 P..."


In [20]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
